In [3]:
!pip install sunpy

In [0]:
import numpy as np
import pandas as pd
import pickle
import sunpy.map
from sunpy.coordinates import frames
from astropy.coordinates import SkyCoord
import astropy.units as u
from scipy.ndimage import zoom
#import Network as net
from keras.utils import to_categorical
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from keras.models import Sequential, load_model
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout
from keras import optimizers

In [0]:
#Import data
with open('output/cnndata/SimpleModel/z-score_model/fourclass/4class_data_model3.pickle','rb') as t:
	 input_train, target_train, input_test, target_test = pickle.load(t)

In [0]:
# Model configuration
batch_size = 32
img_width, img_height, img_num_channels = 250, 250, 1
no_classes = 4
no_epochs = 10
validation_split = 0.4
verbosity = 1
num_folds = 5
input_shape = (img_width, img_height, img_num_channels)

In [0]:
# Define per-fold acc/loss lists
acc_per_fold = []
loss_per_fold = []

#Define training set
inputs = input_train
targets = target_train

In [0]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
histories = []
for train, test in kfold.split(inputs, targets):  
# Define the model architecture  
	model = Sequential()  
	model.add(Convolution2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
	model.add(MaxPooling2D(pool_size=(2, 2)))  
	model.add(Convolution2D(64, kernel_size=(3, 3), activation='relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Flatten())
	model.add(Dropout(0.5))
	model.add(Dense(128, activation='relu'))
	model.add(Dense(4, activation='softmax'))
	sgd = sgd_op(lr=0.001, clipval=0.5)
	model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
	# Fit data to model
	history = model.fit(inputs[train], targets[train], batch_size=batch_size, epochs=no_epochs, verbose=verbosity, validation_split=validation_split)
	histories.append(history)
	# Generate generalization metrics
	scores = model.evaluate(inputs[test], targets[test], verbose=0)
	print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
	acc_per_fold.append(scores[1] * 100)
	loss_per_fold.append(scores[0])
	# Increase fold number
	fold_no = fold_no + 1
	# == Provide average scores ==
	print('------------------------------')
	print('Score per fold')
for i in range(0, len(acc_per_fold)):
	print('------------------------------')
	print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
	print('------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('-------------------------------')